In [6]:
#| default_exp training.transcription.train

In [7]:
#| hide
from nbdev.showdoc import *

In [8]:
#| export
import os
from pathlib import Path
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from torchtext.vocab import vocab, Vocab
from sklearn.model_selection import train_test_split
import math
from timeit import default_timer as timer
from tqdm import tqdm
import numpy as np
import pandas as pd
from collections import OrderedDict, Counter
from multiprocessing import Pool

from llm_mito_scanner.data.download import load_config, \
    get_latest_assembly_path
from llm_mito_scanner.training.transcription.generation import \
    BOS_TOK, EOS_TOK, UNK_TOK, PAD_TOK


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
#| hide
config = load_config()

In [10]:
#| hide
data_path = Path(config.get("data_path"))
data_raw_path = data_path / "raw"
assemblies_path = data_raw_path / "assemblies"
latest_assembly_path = get_latest_assembly_path(assemblies_path)
chromosomes_path = latest_assembly_path / "chromosomes"
training_data_path = latest_assembly_path / "training"
transcription_data_path = training_data_path / "transcription"
sequences_data_path = transcription_data_path / "sequences"

random_state = 42
epochs = 10

In [11]:
#| hide
training_index = pd.read_csv(transcription_data_path / "training_sequence_idx.csv")
batch_idx = []
for i, f in enumerate(np.array_split(training_index, epochs)):
    training_index.loc[f.index.values.tolist(), 'epoch'] = int(i)
training_index.head()

,chromosome,geneid,transcriptid,start,end,type,epoch
0,NC_000019.10,GeneID:946,XM_011527538.4,57,121,intron-small,0.0
1,NC_000006.12,GeneID:124901227,XM_047419609.1,0,64,intron-small,0.0
2,NC_000015.10,GeneID:124903566,XM_047433427.1,7772,7836,intron-small,0.0
3,NC_000002.12,GeneID:375318,NM_198998.3,139,203,intron-small,0.0
4,NC_000006.12,GeneID:55173,XM_017010996.2,3550,3614,intron-small,0.0


In [12]:
#| hide
# Get vocab
training_sequences = list(sequences_data_path.glob("epoch-*/batch-*"))
training_sequences[0]

Path('/mnt/e/Data/llm-mito-scanner-data/data/raw/assemblies/GCF_000001405.40_GRCh38.p14/training/transcription/sequences/epoch-1/batch-1')

In [13]:
#| hide
example_training_sequences = pd.read_parquet(training_sequences[0])
example_training_sequences.loc[:, 'epoch'] = int(str(training_sequences[0]).rsplit("/", 2)[-2].replace("epoch-",""))
example_training_sequences.loc[:, 'batch'] = int(str(training_sequences[0]).rsplit("/", 2)[-1].replace("batch-",""))
example_training_sequences.head()

,input,target,epoch,batch
0,"T,T,T,T,T,G,T,T,T,G,T,T,T,G,T,T,T,G,T,T,T,T,G,...","<intron>,<intron>,<intron>,<intron>,<intron>,<...",1,1
1,"A,T,A,A,A,T,A,A,C,A,T,T,T,G,G,C,T,T,T,A,A,A,A,...","<intron>,<intron>,<intron>,<intron>,<intron>,<...",1,1
2,"A,A,A,A,T,A,A,A,T,G,A,A,T,T,T,T,C,T,T,A,T,C,A,...","<intron>,<intron>,<intron>,<intron>,<intron>,<...",1,1
3,"T,A,G,G,T,G,A,A,T,T,A,T,T,T,T,C,T,T,C,T,C,A,C,...","<intron>,<intron>,<intron>,<intron>,<intron>,<...",1,1
4,"T,A,C,C,T,T,T,C,T,A,C,C,C,A,A,G,A,T,G,C,C,A,C,...","<intron>,<intron>,<intron>,<intron>,<intron>,<...",1,1


In [14]:
#| export
def count_tokens_in_sequence_file(path: Path) -> Counter:
    token_counter = Counter()
    for col in ['input', 'target']:
        sequences = pd.read_parquet(path, columns=[col])[col].str.split(",").tolist()
        sequences_counters = list(map(Counter, sequences))
        token_counter = token_counter + sum(sequences_counters, Counter())
    return token_counter


def build_vocab(transcription_data_path: Path, special_tokens: list[str] = [UNK_TOK, BOS_TOK, EOS_TOK, PAD_TOK]) -> Vocab:
    sequences_path = transcription_data_path / "sequences"
    parquet_files = list(sequences_path.glob("epoch-*/batch-*"))
    pbar = tqdm(total=len(parquet_files), ncols=80, leave=False, desc="Building Vocab")
    pool = Pool(os.cpu_count() - 1)
    token_counter = Counter()
    try:
        for c in pool.imap_unordered(count_tokens_in_sequence_file, parquet_files):
            token_counter = token_counter + c
            pbar.update(1)
    except Exception as e:
        raise e
    finally:
        pool.close()
        pbar.close()
    token_vocab = vocab(OrderedDict(token_counter.most_common()), specials=special_tokens)
    token_vocab.set_default_index(token_vocab[UNK_TOK])
    return token_vocab


def get_vocab(transcription_data_path: Path, special_tokens: list[str] = [UNK_TOK, BOS_TOK, EOS_TOK, PAD_TOK]) -> Vocab:
    vocab_path = transcription_data_path / "vocab.pt"
    if vocab_path.exists():
        token_vocab = torch.load(vocab_path)
        return token_vocab
    else:
        token_vocab = build_vocab(transcription_data_path, special_tokens)
        torch.save(token_vocab, vocab_path)
        return token_vocab

In [15]:
#| hide
training_vocab = get_vocab(transcription_data_path)

In [16]:
#| hide
len(training_vocab)

13

In [17]:
#| hide
training_vocab["<intron>"]

4

In [18]:
#| export
UNK_IDX = None
PAD_IDX = None
BOS_IDX = None
EOS_IDX = None

def set_vocab_idx(vocab: Vocab):
    global UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX
    UNK_IDX = vocab[UNK_TOK]
    PAD_IDX = vocab[PAD_TOK]
    BOS_IDX = vocab[BOS_TOK]
    EOS_IDX = vocab[EOS_TOK]

In [19]:
#| hide
training_vocab[UNK_TOK]

0

In [20]:
#| hide
set_vocab_idx(training_vocab)

In [21]:
#| export
class TranscriptionDataset(Dataset):
    def __init__(self, paths: list[Path], train: bool, **train_test_split_kwargs) -> None:
        self.paths = paths
        self.train = train
        data_train, data_test = train_test_split(pd.read_parquet(paths), **train_test_split_kwargs)
        self.data = data_train if train else data_test
    
    def __len__(self) -> int:
        return self.data.shape[0]

    def __getitem__(self, idx) -> tuple[str, str, int]:
        row = self.data.iloc[idx]
        input_str = row.input.split(",")
        target_str = row.target.split(",")
        return input_str, target_str

In [22]:
#| hide
example_epoch = 1
example_epoch_paths = list((sequences_data_path / f"epoch-{example_epoch}").glob("*batch-*"))

In [23]:
#| hide
# Get train, test datasets
transcription_dataset_train = TranscriptionDataset(example_epoch_paths, True)
transcription_dataset_test = TranscriptionDataset(example_epoch_paths, False)

In [24]:
#| hide
example_input, example_target = transcription_dataset_train[0]
[len(x) if type(x) == list else x for x in transcription_dataset_train[0]]

[64, 64]

## Define model

In [25]:
#| export
# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [26]:
#| hide
torch.manual_seed(random_state)

SRC_VOCAB_SIZE = TGT_VOCAB_SIZE = len(training_vocab)
EMB_SIZE = 128
NHEAD = 4
FFN_HID_DIM = 128
BATCH_SIZE = 64
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [27]:
#| export
# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: list[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

In [28]:
#| export
text_transform = None
def get_text_transform(
        text_vocab: Vocab = None):
    global text_transform
    if text_transform is None:
        if text_vocab is None:
            raise ValueError("Vocab must be passed in text transform not set")
        text_transform = sequential_transforms(
        text_vocab, # Numericalization
        tensor_transform)
    return text_transform

In [29]:
#| hide
get_text_transform(training_vocab)

<function __main__.sequential_transforms.<locals>.func(txt_input)>

In [30]:
#| hide
example_input_transformed = get_text_transform()(example_input)
example_target_transformed = get_text_transform()(example_target)
display(example_input_transformed)
training_vocab.lookup_token(8)

tensor([1, 7, 5, 7, 8, 8, 7, 8, 5, 6, 8, 5, 6, 7, 5, 6, 8, 6, 8, 7, 8, 8, 8, 7,
        8, 5, 7, 8, 8, 6, 6, 5, 8, 7, 6, 8, 6, 8, 8, 5, 8, 8, 8, 8, 5, 8, 6, 6,
        7, 7, 5, 5, 5, 7, 5, 5, 6, 5, 8, 7, 5, 5, 5, 7, 5, 2])

'T'

In [31]:
#| export
# function to collate data samples into batch tensors
def collate_fn(batch: list[tuple[str, str, int]]):
    global PAD_IDX
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(get_text_transform()(src_sample))
        tgt_batch.append(get_text_transform()(tgt_sample))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [32]:
#| hide
example_loader = DataLoader(transcription_dataset_train, batch_size=32, collate_fn=collate_fn)
example_batch = next(iter(example_loader))

In [33]:
#| hide
len(example_batch)

2

In [34]:
#| hide
example_batch[0].shape

torch.Size([66, 32])

In [35]:
#| hide
training_vocab.lookup_token(0)

'<unk>'

In [36]:
#| hide
try:
    del model
except:
    pass
torch.cuda.empty_cache()
model = transformer
model.train()
losses = 0

In [37]:
#| hide
batch_loss = None

In [38]:
#| hide
chunks_attached = False

In [39]:
#| hide
if not chunks_attached:
    example_input = example_batch[0].to(DEVICE)
    example_target = example_batch[1].to(DEVICE)
    chunks_attached = True

In [40]:
#| hide
example_target_input = example_target[:-1, :]

In [41]:
#| hide
example_target_out = example_target[1:, :]

In [42]:
#| hide
example_input.shape, example_target.shape, example_target_input.shape, example_target_out.shape

(torch.Size([66, 32]),
 torch.Size([66, 32]),
 torch.Size([65, 32]),
 torch.Size([65, 32]))

In [43]:
#| export
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [44]:
#| hide
masks_made = False

In [45]:
#| hide
# del example_src_mask, example_tgt_mask, example_src_padding_mask, example_tgt_padding_mask

In [46]:
#| hide
if not masks_made:
    example_src_mask, example_tgt_mask, example_src_padding_mask, example_tgt_padding_mask = create_mask(
        example_input, 
        example_target_input
        )
    masks_made = True

for t in [example_src_mask, example_tgt_mask, example_src_padding_mask, example_tgt_padding_mask]:
    print(t.shape)

torch.Size([66, 66])
torch.Size([65, 65])
torch.Size([32, 66])
torch.Size([32, 65])


In [47]:
#| hide
logits = model(
    example_input, 
    example_target_input, 
    example_src_mask, 
    example_tgt_mask,
    example_src_padding_mask, 
    example_tgt_padding_mask, 
    example_src_padding_mask
    )

In [48]:
#| hide
optimizer.zero_grad()

loss = loss_fn(logits.reshape(-1, logits.shape[-1]), example_target_out.reshape(-1))
# loss.backward()
# optimizer.step()
# losses += loss.item()

In [49]:
#| hide
loss

tensor(3.2941, device='cuda:0', grad_fn=<NllLossBackward0>)

In [50]:
#| hide
torch.cuda.empty_cache()
try:
    for var in [
        model,
        optimizer,
        loss,
        batch_loss,
        example_input,
        example_target,
        example_target_input,
        example_target_out,
        example_src_mask, 
        example_tgt_mask,
        example_src_padding_mask, 
        example_tgt_padding_mask,
    ]:
        try:
            del var
        except Exception:
            pass
except NameError as e:
    raise e

In [51]:
#| hide
len(example_loader), len(transcription_dataset_train) // 32

(6059, 6058)

In [52]:
#| export
def train_epoch(
        model, optimizer, loss_fn, 
        dataset: TranscriptionDataset,
        batch_size: int = 32,
        pbar_position: int = 1,
        limit: int = None):
    global DEVICE
    model.train()
    losses = 0
    train_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)
    batches = len(train_dataloader)
    batch_pbar = tqdm(total=batches, position=pbar_position, leave=False, ncols=80, desc="Training")
    for i, (src, tgt) in enumerate(train_dataloader):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        batch_pbar.update(1)
        if isinstance(limit, int):
            if i >= limit:
                batches = limit
                break

    batch_pbar.close()

    return losses / batches


def evaluate(
        model, loss_fn, 
        dataset: TranscriptionDataset,
        batch_size: int = 32,
        pbar_position: int = 1,
        limit: int = None):
    model.eval()
    losses = 0

    val_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)
    batches = len(val_dataloader)
    eval_pbar = tqdm(total=batches, position=pbar_position, leave=False, ncols=80, desc="Evaluating")
    for i, (src, tgt) in enumerate(val_dataloader):
        # Break these up into reasonable sizes for the GPU
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
        
        eval_pbar.update(1)
        if isinstance(limit, int):
            if i >= limit:
                batches = limit
                break
        
    eval_pbar.close()
    return losses / batches

In [53]:
#| hide
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

example_train_loss = train_epoch(
    transformer, optimizer, loss_fn,
    transcription_dataset_train,
    batch_size = 32,
    pbar_position = 0,
    limit=4)

example_eval_loss = evaluate(
    transformer, loss_fn, 
    transcription_dataset_test,
    batch_size = 32,
    pbar_position = 0,
    limit=4)

example_train_loss, example_eval_loss

(3.1575343012809753, 1.6769118010997772)

In [55]:
#| export
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, vocab: Vocab, src_sentence: str, BOS_TOKEN: str = BOS_TOK, sep: str = ","):
    model.eval()
    src = get_text_transform(vocab)(src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    bos_idx = vocab[BOS_TOKEN]
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=bos_idx).flatten()
    return sep.join(
        vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))
    ).replace("<bos>", "").replace("<eos>", "")

In [56]:
#| hide
example_sequence = list("AGTGCCGTAA")
example_translation = translate(transformer, training_vocab, example_sequence)
example_translation

' <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron>'

In [2]:
#| hide
import nbdev; nbdev.nbdev_export()